In [93]:
import fastf1
import pandas as pd
import os
from matplotlib import pyplot as plt
import numpy as np
import warnings
from pathlib import Path
import seaborn as sns
import logging

warnings.filterwarnings('ignore')
logging.getLogger('fastf1').setLevel(logging.WARNING)

fastf1.Cache.enable_cache(os.getcwd()) # replace with ur own folder path to make a cache for faster loading

# Window size (range of the sliding window used in delta angle calculation)
N = 15 # (40 + 1 + 40  = 81 data points)

# Max and Min delta angle for a data point to be classified as being part of a corner
CORNER_THRESHOLD = 0.4

In [95]:
year = 2025

gp_events = ['British Grand Prix', 'Belgian Grand Prix', 'Hungarian Grand Prix', 'Dutch Grand Prix', 'Emilia Romagna Grand Prix', 'Monaco Grand Prix']

df_f1 = pd.DataFrame()
drivers = []
teams = []
events = []

for event in gp_events:
    session = fastf1.get_session(year, event, 'R')
    session.load()

    current_drivers = [session.get_driver(num)['Abbreviation'] for num in session.drivers]
    current_teams = [session.get_driver(num)['TeamName'] for num in session.drivers]

    drivers.extend(current_drivers)
    teams.extend(current_teams)
    events.extend([event] * len(current_drivers))

df_f1["Driver"] = drivers
df_f1["Team"] = teams
df_f1["Event"] = events

df_f1.to_csv("f1.csv", index=False)

core        WARNING 	Fixed incorrect tyre stint information for driver '81'
core        WARNING 	Fixed incorrect tyre stint information for driver '4'
core        WARNING 	Fixed incorrect tyre stint information for driver '16'
core        WARNING 	Fixed incorrect tyre stint information for driver '1'
core        WARNING 	Fixed incorrect tyre stint information for driver '63'
core        WARNING 	Fixed incorrect tyre stint information for driver '23'
core        WARNING 	Fixed incorrect tyre stint information for driver '44'
core        WARNING 	Fixed incorrect tyre stint information for driver '30'
core        WARNING 	Fixed incorrect tyre stint information for driver '5'
core        WARNING 	Fixed incorrect tyre stint information for driver '10'
core        WARNING 	Fixed incorrect tyre stint information for driver '87'
core        WARNING 	Fixed incorrect tyre stint information for driver '27'
core        WARNING 	Fixed incorrect tyre stint information for driver '22'
core        WAR

In [97]:
race_stats = pd.DataFrame()
race_data = []

for event in df_f1['Event'].unique():
    session = fastf1.get_session(2025, event, 'R')
    session.load()

    telemetry = session.laps.pick_driver('VER').get_telemetry()
    telemetry = telemetry[['X', 'Y','Speed']]

    dx1 = telemetry['X'] - telemetry['X'].shift(N)
    dy1 = telemetry['Y'] - telemetry['Y'].shift(N)
    dx2 = telemetry['X'].shift(-N) - telemetry['X']
    dy2 = telemetry['Y'].shift(-N) - telemetry['Y']

    telemetry['Event'] = event

    telemetry['Delta_Angle'] = np.arctan2(dx1*dy2 - dy1*dx2, dx1*dx2 + dy1*dy2)
    telemetry['Delta_Angle'] = telemetry['Delta_Angle'].fillna(0)

    telemetry['IsCorner'] = ((telemetry['Delta_Angle'] > CORNER_THRESHOLD) | (telemetry['Delta_Angle'] < -CORNER_THRESHOLD) | (telemetry['Speed'] < 180))

    race_data.append(telemetry)

race_stats = pd.concat(race_data, ignore_index=True)
race_stats.to_csv("race_stats.csv", index=False)

core        WARNING 	Fixed incorrect tyre stint information for driver '81'
core        WARNING 	Fixed incorrect tyre stint information for driver '4'
core        WARNING 	Fixed incorrect tyre stint information for driver '16'
core        WARNING 	Fixed incorrect tyre stint information for driver '1'
core        WARNING 	Fixed incorrect tyre stint information for driver '63'
core        WARNING 	Fixed incorrect tyre stint information for driver '23'
core        WARNING 	Fixed incorrect tyre stint information for driver '44'
core        WARNING 	Fixed incorrect tyre stint information for driver '30'
core        WARNING 	Fixed incorrect tyre stint information for driver '5'
core        WARNING 	Fixed incorrect tyre stint information for driver '10'
core        WARNING 	Fixed incorrect tyre stint information for driver '87'
core        WARNING 	Fixed incorrect tyre stint information for driver '27'
core        WARNING 	Fixed incorrect tyre stint information for driver '22'
core        WAR

In [98]:
race_stats_csv = pd.read_csv("race_stats.csv")

for event in race_stats_csv["Event"].unique():
    event_data = race_stats_csv[race_stats_csv["Event"] == event]

    plt.figure(figsize=(6.4, 4.8))
    cmap = sns.color_palette('viridis', as_cmap = True)
    norm = plt.Normalize(vmin=race_stats_csv['Delta_Angle'].min(), vmax=race_stats_csv['Delta_Angle'].max())

    sc = sns.scatterplot(data=event_data,
                        x='X', y='Y',
                        hue='Delta_Angle',
                        palette='viridis',
                        hue_norm=norm,
                        legend=False,
                        edgecolor=None)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax = plt.gca())

    plt.savefig(f"{event}_deltaAngle.png")
    plt.close()

    plt.figure(figsize=(6.4, 4.8))
    sns.scatterplot(data=event_data,
                    x='X', y='Y',
                    hue='IsCorner',
                    palette={False: "blue", True: "red"},
                    edgecolor=None)

    plt.savefig(f"{event}_isCorner.png")
    plt.close()

In [33]:
#proposed df builder

df = pd.read_csv('f1.csv')

new_cols = ['TyreCompound','Top Speed','Qualifying Time',
            'AvgCornerSpeed','AvgCornerEntrySpeed','AvgCornerExitSpeed',
            'AvgStraightSpeed','AvgStraightEntrySpeed','AvgStraightExitSpeed',
            'AirTemp','Humidity','Rainfall','TrackTemp',
            'WindDirection','WindSpeed']

for col in new_cols:
    df[col] = None

for i, row in df.iterrows():
    driver_abr = row['Driver']
    event = row['Event']

    session = fastf1.get_session(year, event, 'R')
    session.load()

    driver_laps = session.laps.pick_driver(driver_abr)
    if driver_laps.empty:
        continue

    lap = driver_laps.pick_fastest()
    tel = lap.get_car_data()

    # Filling in data
    df.at[i, 'TyreCompound'] = lap['Compound'] if 'Compound' in lap else None
    df.at[i, 'Qualifying Time'] = lap['LapTime'].total_seconds() if lap['LapTime'] else None
    df.at[i, 'Top Speed'] = tel['Speed'].max()

    # TODO: Define criteria for the speeds
    corner_speeds = tel.query('Speed < 200')['Speed']
    straight_speeds = tel.query('Speed >= 200')['Speed']

    # TODO: Average calculations for speed...How many should we sample from
    df.at[i, 'AvgCornerSpeed'] = corner_speeds.mean() if not corner_speeds.empty else None
    df.at[i, 'AvgCornerEntrySpeed'] = corner_speeds.head(5).mean() if len(corner_speeds) >= 5 else None
    df.at[i, 'AvgCornerExitSpeed'] = corner_speeds.tail(5).mean() if len(corner_speeds) >= 5 else None
    df.at[i, 'AvgStraightSpeed'] = straight_speeds.mean()if not straight_speeds.empty else None
    df.at[i, 'AvgStraightEntrySpeed'] = straight_speeds.head(5).mean() if len(straight_speeds) >= 5 else None
    df.at[i, 'AvgStraightExitSpeed'] = straight_speeds.tail(5).mean() if len(straight_speeds) >= 5 else None

    # Weather
    weather = session.weather_data
    df.at[i, 'AirTemp'] = weather['AirTemp'].mean()
    df.at[i, 'Humidity'] = weather['Humidity'].mean()
    df.at[i, 'Rainfall'] = weather['Rainfall'].max()
    df.at[i, 'TrackTemp'] = weather['TrackTemp'].mean()
    df.at[i, 'WindDirection'] = weather['WindDirection'].mean()
    df.at[i, 'WindSpeed'] = weather['WindSpeed'].mean()

df.to_csv('f1.csv', index = False)


KeyboardInterrupt

